# * VINSIGHT : Data Issue

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Issue Transaction

In [3]:
''' Active Caller '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250401
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD
        , AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = 'TB1S000600' --Prepaid Active Caller : TMH
    AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    # print(f'\nCurrent Timestamp...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    txn_df = chk_src_df.copy()
    mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    for col in mod_col_list:
        txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-04-23, 11:10:50
   -> v_start_date: 20250401

    TM_KEY_MTH  TM_KEY_DAY   METRIC_CD                              METRIC_NAME   AREA_NAME  ACTUAL_AS_OF ACTUAL_SNAP ACTUAL_AGG_MTH              PPN_TM
0       202504    20250401  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250401  17,205,721     17,205,721 2025-04-23 00:34:11
1       202504    20250402  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250402  17,200,601     17,200,601 2025-04-23 00:34:11
2       202504    20250403  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250403  17,198,038     17,198,038 2025-04-23 00:34:11
3       202504    20250404  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250404  17,166,314     17,166,314 2025-04-23 00:34:11
4       202504    20250405  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250405  17,132,397     17,132,397 2025-04-23 00

In [4]:
''' 080 : DTAC '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME
        , SUM(CASE WHEN AREA_CD = '080' THEN ACTUAL_SNAP END) ACTUAL_080
        , SUM(CASE WHEN AREA_CD = '080' THEN TARGET_SNAP END) TARGET_080
        , SUM(CASE WHEN AREA_CD = '016' THEN ACTUAL_SNAP END) ACTUAL_016
        , SUM(CASE WHEN AREA_CD = '016' THEN TARGET_SNAP END) TARGET_016
        , SUM(CASE WHEN AREA_CD = '040' THEN ACTUAL_SNAP END) ACTUAL_040
        , SUM(CASE WHEN AREA_CD = '040' THEN TARGET_SNAP END) TARGET_040
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF, MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = 'DB1R000100' --Prepaid Revenue : DTAC
    --WHERE METRIC_CD = 'DB2R000100' --Postpaid Revenue : DTAC
    --WHERE METRIC_CD = 'DB2R010100' --Postpaid Revenue B2C : DTAC
    --WHERE METRIC_CD = 'DB2R020100' --Postpaid Revenue B2B : DTAC
    AND AREA_CD IN ('016', '040', '080')
    AND TM_KEY_DAY >= {v_start_date} 
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME
    ORDER BY TM_KEY_DAY, METRIC_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    txn_df = chk_src_df.copy()
    mod_col_list = txn_df.iloc[:, 4:10].columns.tolist()
    for col in mod_col_list:
        txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{txn_df.to_string(max_cols=100)}')
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-04-23, 11:11:05
   -> v_start_date: 20250101

     TM_KEY_MTH  TM_KEY_DAY   METRIC_CD             METRIC_NAME ACTUAL_080 TARGET_080 ACTUAL_016 TARGET_016 ACTUAL_040 TARGET_040  ACTUAL_AS_OF              PPN_TM
0        202501    20250101  DB1R000100  Prepaid Revenue : DTAC  1,458,637        nan    362,185    365,268  1,096,452  1,060,020      20250101 2025-04-23 02:12:31
1        202501    20250102  DB1R000100  Prepaid Revenue : DTAC  1,378,370        nan    365,318    373,482  1,013,052  1,083,855      20250102 2025-04-23 02:12:31
2        202501    20250103  DB1R000100  Prepaid Revenue : DTAC  1,330,136        nan    356,411    362,361    973,725  1,051,581      20250103 2025-04-23 02:12:31
3        202501    20250104  DB1R000100  Prepaid Revenue : DTAC  1,229,670        nan    312,077    348,242    917,593  1,010,608      20250104 2025-04-23 02:12:31
4        202501    20250105  DB1R000100  Prepaid Revenue : DTAC  1,312,542        nan    32

In [2]:
''' Prepaid Revenue : TMH '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250401
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD
        , AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = 'TB1R000100' --Prepaid Revenue : TMH
    AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    txn_df = chk_src_df.copy()
    mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    for col in mod_col_list:
        txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-04-24, 10:56:28
   -> v_start_date: 20250401

    TM_KEY_MTH  TM_KEY_DAY   METRIC_CD            METRIC_NAME   AREA_NAME  ACTUAL_AS_OF  ACTUAL_SNAP ACTUAL_AGG_MTH              PPN_TM
0       202504    20250401  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250401  101,725,829    101,725,829 2025-04-23 23:05:38
1       202504    20250402  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250402   92,314,263    194,040,092 2025-04-23 23:05:38
2       202504    20250403  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250403   85,596,819    279,636,911 2025-04-23 23:05:38
3       202504    20250404  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250404   85,954,445    365,591,356 2025-04-23 23:05:38
4       202504    20250405  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250405   84,407,001    449,998,357 2025-04-23 23:05:38
5       202504    20250406  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250406   82,55